In [ ]:
%pip install -U bitsandbytes
%pip install -U transformers
%pip install -U peft
%pip install -U accelerate
%pip install -U trl 
print("Done")

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os,torch, wandb
from datasets import load_dataset
from trl import SFTTrainer

c:\Users\Admin\miniconda3\lib\site-packages\transformers\utils\hub.py:128: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [2]:
!huggingface-cli login 

^C


In [2]:
# Monitering the LLM
wandb.login(key = "ec7cc0ef28b4a745b45ed281f881bfee6399aed7")
run = wandb.init(
    project='Fine tuning mistral 7B', 
    job_type="training", 
    anonymous="allow"
)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: bonpaul (bonpaul-tr-ng-i-h-c-khoa-h-c-t-nhi-n-hqg-hcm). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\Admin\_netrc


In [3]:
base_model = "meta-math/MetaMath-Mistral-7B"  # Hugging Face model repo
#dataset_name = "tranthaihoa/math_test"
new_model = "mistral_7b_eng-math"

In [8]:
from datasets import load_dataset, Dataset
from pathlib import Path

# Dataset details
dataset_name = "tranthaihoa/math_test"
output_dir = Path("D:\cache")  # Specify output directory for saving datasets

# Preprocessing function
def preprocess_meta_math(batch):
    result = {
        "instruction": [],
        "response": []
    }
    
    for i in range(len(batch["id"])):
        # Construct instruction and response
        instruction = (
            f"[ID: {batch['id'][i]}] "
            f"[Question: {batch['Question'][i]}] "
            f"[Explanation: {batch['Explanation'][i]}] "
            f"[Inference Steps: {batch['Inference Steps'][i]}] "
            f"[Grade: {batch['Grade'][i]}] "
            f"[Source: {batch['Source'][i]}] "
            f"[Instruction: {batch['Instruction'][i]}] "
            f"[Response Type: {batch['Response Type'][i]}] "
            f"[Math Type: {batch['Math Type'][i]}]"
        )
        response = batch["Answer"][i]

        result["instruction"].append(instruction)
        result["response"].append(response)
    
    return result

# Load and preprocess the dataset
ds = load_dataset(dataset_name)
preprocessed_ds = ds.map(preprocess_meta_math, batched=True, remove_columns=ds["train"].column_names)

# Shuffle the dataset
shuffled_dataset = preprocessed_ds["train"].shuffle(seed=42)

# Limit to the total rows in the dataset
total_rows = 821
limited_dataset = shuffled_dataset.select(range(total_rows))

# Calculate split sizes (80% train, 10% validation, 10% test)
train_size = int(0.8 * total_rows)  # 656 rows for train
val_size = int(0.1 * total_rows)    # 82 rows for validation
test_size = total_rows - train_size - val_size  # 83 rows for test

# Split the dataset
train_dataset = limited_dataset.select(range(train_size))
val_dataset = limited_dataset.select(range(train_size, train_size + val_size))
test_dataset = limited_dataset.select(range(train_size + val_size, total_rows))

# Print the sizes of each split
print(f"Total dataset size: {total_rows}")
print(f"Train set size: {len(train_dataset)}")
print(f"Validation set size: {len(val_dataset)}")
print(f"Test set size: {len(test_dataset)}")

# Save splits in Hugging Face Dataset format
train_dataset.save_to_disk("D:/cache/train_dataset")
val_dataset.save_to_disk("D:/cache/validation_dataset")
test_dataset.save_to_disk("D:/cache/test_dataset")

print(f"Datasets saved to {output_dir}")


Total dataset size: 821
Train set size: 656
Validation set size: 82
Test set size: 83


Saving the dataset (0/1 shards):   0%|          | 0/656 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/82 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/83 [00:00<?, ? examples/s]

Datasets saved to D:\cache


FIXXXXXXXXX

In [4]:
from datasets import load_from_disk


train_dataset = load_from_disk("D:/cache/translated_train_dataset")
val_dataset = load_from_disk("D:/cache/translated_validation_dataset")
test_dataset = load_from_disk("D:/cache/translated_test_dataset")

In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from safetensors.torch import load_file  # Import for loading safetensors

# Configuration for 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Define the base model and adapter checkpoint paths
base_model_name = base_model  # Replace with the actual base model name
#adapter_checkpoint_path = "D:/Chatbot-Project/fine-tuned-model/checkpoint-2500/adapter_model.safetensors"

# Load the base model
model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

# Load adapter weights using safetensors
#adapter_state_dict = load_file(adapter_checkpoint_path)
#model.load_state_dict(adapter_state_dict, strict=False)  # Load adapter into the model

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

print("Model and tokenizer loaded successfully!")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Model and tokenizer loaded successfully!


In [6]:
tokenizer = AutoTokenizer.from_pretrained(
    base_model,
    padding_side="left",
    add_eos_token=True,
    add_bos_token=True,
)
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):

    return tokenizer(
        examples["instruction"],  # Use preformatted 'instruction' as the input
        text_pair=examples["response"],  # Use 'response' for paired input-output sequences
        padding="max_length",  # Pad sequences to the maximum length
        truncation=True,       # Truncate sequences longer than the max length
        max_length=512,        # Set the maximum length for tokenization
    )

# Tokenize the train and validation datasets
tokenized_train_dataset = train_dataset.map(
    tokenize_function, 
    batched=True  # Process multiple examples at once for efficiency
)

tokenized_val_dataset = val_dataset.map(
    tokenize_function, 
    batched=True  # Similarly process the validation dataset
)

Map:   0%|          | 0/656 [00:00<?, ? examples/s]

Map:   0%|          | 0/82 [00:00<?, ? examples/s]

In [7]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

print(model)


MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32001, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNo

In [8]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=32,
    lora_alpha=64,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head",
    ],
    bias="none",
    lora_dropout=0.05,  # Conventional
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, config)
print_trainable_parameters(model)


trainable params: 85041184 || all params: 3837120544 || trainable%: 2.216276059739029


In [9]:
if torch.cuda.device_count() > 1: # If more than 1 GPU
    model.is_parallelizable = True
    model.model_parallel = True

In [10]:
from accelerate import Accelerator

# Initialize the accelerator
accelerator = Accelerator()
model = accelerator.prepare_model(model)

In [11]:
from pathlib import Path
from transformers import Trainer, TrainingArguments, TrainerCallback
import transformers
import torch
from datetime import datetime
import shutil
import os

# Paths and project configuration
cache_dir = Path("D:/cache")  # Kaggle's writable directory
project = "eng-math-finetune"
base_model_name = "mistral"
run_name = f"{base_model_name}-{project}"
output_dir = cache_dir / f"mistral-fine-tuned-{run_name}"

# Ensure the output directory exists
output_dir.mkdir(parents=True, exist_ok=True)

# Dataset parameters
num_train_samples = 656  # Train set size
num_val_samples = 82     # Validation set size
num_test_samples = 83    # Test set size
batch_size = 4           # Set a larger batch size
num_epochs = 6           # Set number of epochs to 6

# Calculate steps
steps_per_epoch = (num_train_samples + batch_size - 1) // batch_size  # Round up
total_steps = steps_per_epoch * num_epochs
print("steps per epoch:", steps_per_epoch)

# Define optimizer with weight decay
optimizer = torch.optim.AdamW(model.parameters(), lr=2.5e-5, weight_decay=0.1)

# Define trainer
trainer = Trainer(
    model=model,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    args=TrainingArguments(
        output_dir=output_dir,
        warmup_steps=100,  # Adjust warmup steps to fit the smaller dataset
        per_device_train_batch_size=batch_size,
        gradient_accumulation_steps=1,
        gradient_checkpointing=True,
        max_steps=total_steps,  # Set calculated total steps
        learning_rate=2.5e-5,
        weight_decay=0.1,  # Apply weight decay
        bf16=True,  # Enable bf16 if supported
        optim="paged_adamw_8bit",
        logging_steps=steps_per_epoch,  # Log once per epoch
        logging_dir=str(cache_dir / "logs"),
        save_strategy="steps",
        save_steps=steps_per_epoch,  # Save every epoch
        evaluation_strategy="steps",
        eval_steps=steps_per_epoch,  # Evaluate every epoch
        do_eval=True,
        report_to="wandb",
        run_name=f"{run_name}-{datetime.now().strftime('%Y-%m-%d-%H-%M')}"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
    optimizers=(optimizer, None),  # Pass the optimizer with weight decay
)

# Disable cache for training
model.config.use_cache = False

# Start training
trainer.train()

print(f"Model and tokenizer have been saved to: {output_dir}")

steps per epoch: 164


c:\Users\Admin\miniconda3\lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


  0%|          | 0/984 [00:00<?, ?it/s]

{'loss': 0.9756, 'grad_norm': 2.1959965229034424, 'learning_rate': 2.319004524886878e-05, 'epoch': 1.0}


  0%|          | 0/11 [00:00<?, ?it/s]

...

{'loss': 0.1231, 'grad_norm': 5.296316623687744, 'learning_rate': 0.0, 'epoch': 6.0}


  0%|          | 0/11 [00:00<?, ?it/s]

{'eval_loss': 1.0480365753173828, 'eval_runtime': 39.3995, 'eval_samples_per_second': 2.081, 'eval_steps_per_second': 0.279, 'epoch': 6.0}


c:\Users\Admin\miniconda3\lib\site-packages\peft\utils\save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


{'train_runtime': 6961.0432, 'train_samples_per_second': 0.565, 'train_steps_per_second': 0.141, 'train_loss': 0.4369705188565138, 'epoch': 6.0}
Model and tokenizer have been saved to: D:\cache\mistral-fine-tuned-mistral-eng-math-finetune
